In [113]:
import pandas as pd
from my_connector import MySQLConnector

# Limpieza

In [114]:
# noinspection SpellCheckingInspection
df_padron = pd.read_csv("Datasets/padron22.csv", low_memory=False, encoding='UTF-8')

In [115]:
df_padron.head(20)

Tiramos las columnas que no podemos utilizar para cruzar con accidentalidad. Así como los datos repetidos

In [116]:
df_padron.drop(columns=['Ï»¿EJERICIO', 'DISTRITO', 'BARRIO', 'COD_BARRIO', 'CONTADOR'], inplace=True)

Arreglamos los códigos para que tengan sentido

In [117]:
cod_veh = df_padron['COD_TIPO_VEHICULO'].unique()
tipo_veh = df_padron['TIPO_VEHICULO'].unique()

In [118]:
cod_veh

In [119]:
tipo_veh

In [120]:
for row in zip(cod_veh, tipo_veh):
    print(row)

In [121]:
df_padron['COD_TIPO_VEHICULO'] = df_padron['COD_TIPO_VEHICULO'].replace({'CA': 'C', 'MT': 'M', 'RE': 'O', 'TR': 'T', 'TU': 'T'})

In [122]:
cod_veh = df_padron['COD_TIPO_VEHICULO'].unique()
tipo_veh = df_padron['TIPO_VEHICULO'].unique()
for row in zip(cod_veh, tipo_veh):
    print(row)

In [123]:
df_padron.pop('TIPO_VEHICULO').head()

In [124]:
df_padron.head()

In [125]:
cod_veh = df_padron['ETIQUETA_MEDIOAMBIENTAL'].unique()
tipo_veh = df_padron['CLASIFICACIÃ\x93N_AMBIENTAL'].unique()
for row in zip(cod_veh, tipo_veh):
    print(row)

In [126]:
df_padron['ETIQUETA_MEDIOAMBIENTAL'] = df_padron['ETIQUETA_MEDIOAMBIENTAL'].replace({'--': 'A'})
df_padron['ETIQUETA_MEDIOAMBIENTAL'] = df_padron['ETIQUETA_MEDIOAMBIENTAL'].fillna('A')
df_padron.pop('CLASIFICACIÃ\x93N_AMBIENTAL').head()

In [127]:
df_padron['COD_TIPO_PERSONA'].unique()

In [128]:
df_padron['COD_TIPO_PERSONA'] = df_padron['COD_TIPO_PERSONA'].replace({'E': 'S'})

Solo existe una fila con la O y no tiene una clasificación clara así que lo he tirado

In [129]:
df_padron = df_padron.drop(df_padron[df_padron['COD_TIPO_PERSONA'] == 'O'].index)

In [130]:
df_padron.pop('TIPO_PERSONA').head()

In [131]:
df_padron[df_padron['COD_DISTRITO'] == 0]

In [132]:
df_padron['TIPO_CARBURANTE'].unique()

In [133]:
df_padron['TIPO_CARBURANTE'] = df_padron['TIPO_CARBURANTE'].replace(
    {'HÃBRIDO': 'HIBRIDO', 'ELÃCTRICO': 'ELECTRICO', 'Tipo carburante desconocido': 'DESCONOCIDO'})

In [134]:
df_padron.rename(columns={'AÃ\x91O_MATRICULACIÃ\x93N': 'Y_MATRICULACION'}, inplace=True)

In [135]:
df_padron.head()

Vamos a clusterizar los datos ya que hay muchos repetidos

In [136]:
df_padron.pop('CUOTA').head()

In [137]:
df_padron['OCURRENCIAS'] = 0

In [138]:
vehicles = df_padron.groupby(
    ['COD_TIPO_PERSONA', 'COD_DISTRITO', 'COD_TIPO_VEHICULO', 'ETIQUETA_MEDIOAMBIENTAL', 'TIPO_CARBURANTE',
     'Y_MATRICULACION']).count()

In [139]:
vehicles.reset_index(inplace=True)

In [140]:
vehicles['Y_MATRICULACION'] = vehicles['Y_MATRICULACION'].replace({'Desconocido':-1})

In [141]:
vehicles.head()

In [142]:
connector = MySQLConnector('localhost', 'root', 'root', 'TNT')
connector.connect()

In [143]:
import uuid
i = 0
for index, row in vehicles.iterrows():
    i = i + 1
    try:
        id = uuid.uuid4()
        owner_type = row['COD_TIPO_PERSONA']
        district_code = row['COD_DISTRITO']
        vehicle_type = row['COD_TIPO_VEHICULO']
        sticker = row['ETIQUETA_MEDIOAMBIENTAL']
        fuel_type = row['TIPO_CARBURANTE']
        year_matriculation = row['Y_MATRICULACION']
        num_ocurrence = row['OCURRENCIAS']
    
        # Construir la sentencia SQL INSERT
        insert_query = f"INSERT INTO Vehicles (id, owner_type, district_code, vehicle_type, sticker, fuel_type, year_matriculation, num_ocurrence) VALUES ('{id}', '{owner_type}', {district_code}, '{vehicle_type}', '{sticker}', '{fuel_type}', {year_matriculation}, {num_ocurrence});"
        connector.execute_query(insert_query)
        
        print(f"Inserting row: {index}, item: {i}")
    except Exception as e:
        print(e)


In [144]:
connector.close()